In [1]:
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, LSTM
import numpy as np
import string
import pickle

#### Load beer and brahma references to train the ML

In [3]:
def clean_sentence(sentence):
    sentence = sentence.replace(",", "")
    sentence = sentence.replace(".", "")
    sentence = sentence.replace("-", "")
    sentence = sentence.replace("/", "")
    sentence = sentence.replace(":", "")
    sentence = sentence.replace("!", "")
    sentence = sentence.replace("\"", "")
    sentence = sentence.replace("“","")
    return sentence.lower()
    
corpus = ""
with open('todas_campanhas.txt', 'r') as f:
    datastore = json.load(f)
    for item in datastore:
            corpus += " " + clean_sentence(item['titulo'])

with open('frases_famosos.txt', 'r') as f:
    datastore = json.load(f)
    for item in datastore:
            corpus += " " + clean_sentence(item['frase'])


#### Create sentences for each word in the corpus using the next "window_size" words

In [17]:
window_size=10
max_sequence_len = 30
sentences = []
alltext = []
words = corpus.split(" ")
print("Number words: ", len(words))
range_size = len(words) - max_sequence_len
for i in range(0, range_size):
    thissentence = ""
    for word in range(0, window_size-1):
        word = words[i+word]
        thissentence = thissentence + word
        thissentence = thissentence + " "
    sentences.append(thissentence)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print("Number of unique words: ", total_words)
print("Number of sentences generated: ", len(sentences))

Number words:  1474
Number of unique words:  594
Number of sentences generated:  1444


#### Load the saved model "modelo_refresca"

In [5]:
model = keras.models.load_model("modelo_refresca")
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 4)           2376      
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 58)          7888      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 58)                20416     
_________________________________________________________________
dense_1 (Dense)              (None, 594)               35046     
Total params: 65,726
Trainable params: 65,726
Non-trainable params: 0
_________________________________________________________________
None


#### Function to generate the sentence
seed_text: input text that will be used to generate the next word

next_words: number of words to be generated

number_options: number of top ranked words by the ML

In [21]:
def generate_sentence(seed_text, next_words, number_options=1):
    _seed_text = seed_text
    if next_words == 0:
        print(seed_text)
        return
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([_seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        arr = model.predict(token_list)
        ind = np.argpartition(np.array(arr[0]), -number_options)[-number_options:]
        output_word = ''

        for word, index in tokenizer.word_index.items():
            if index in ind:
                output_word = word
                seed_text_temp = _seed_text
                seed_text_temp += " " + output_word
                generate_sentence(seed_text_temp, (next_words - 1), number_options)
        break;

#### Example to print a sentence with 9 next words and just the first top ranked word

In [22]:
print(generate_sentence("cerveja", 9))

cerveja oficial do nã nã nã nã refresca até pensamento
None


#### Example to print a sentence with 5 next words and 3 top ranked words

In [40]:
print(generate_sentence("brindar com brahma é", 5, 3))

brindar com brahma é a se faz a copa
brindar com brahma é a se faz a mais
brindar com brahma é a se faz a relação
brindar com brahma é a se faz o mundo
brindar com brahma é a se faz o sabor
brindar com brahma é a se faz o festival
brindar com brahma é a se faz sentir cerveja
brindar com brahma é a se faz sentir da
brindar com brahma é a se faz sentir camarote
brindar com brahma é a se alia a a
brindar com brahma é a se alia a seus
brindar com brahma é a se alia a equipe
brindar com brahma é a se alia o chopp
brindar com brahma é a se alia o mundo
brindar com brahma é a se alia o duplo
brindar com brahma é a se alia eu igtv
brindar com brahma é a se alia eu aproveitei
brindar com brahma é a se alia eu superar
brindar com brahma é a se faz” brahma cria
brindar com brahma é a se faz” brahma celebra
brindar com brahma é a se faz” brahma anuncia
brindar com brahma é a se faz” em nova
brindar com brahma é a se faz” em ação
brindar com brahma é a se faz” em tirar
brindar com brahma é a se faz

brindar com brahma é um brinde ao melhor festival
brindar com brahma é um brinde ao melhor seu
brindar com brahma é um brinde ao melhor pai
brindar com brahma é um brinde paulo sentir a
brindar com brahma é um brinde paulo sentir cerveja
brindar com brahma é um brinde paulo sentir especial
brindar com brahma é um brinde paulo deus se
brindar com brahma é um brinde paulo deus não
brindar com brahma é um brinde paulo deus soubesse
brindar com brahma é um brinde paulo tiver uma
brindar com brahma é um brinde paulo tiver ao
brindar com brahma é um brinde paulo tiver sem
brindar com brahma é um país não pode que
brindar com brahma é um país não pode ser
brindar com brahma é um país não pode gaúcho
brindar com brahma é um país não grandes sua
brindar com brahma é um país não grandes astros
brindar com brahma é um país não grandes terá
brindar com brahma é um país não nada mais
brindar com brahma é um país não nada brahmeiro
brindar com brahma é um país não nada até
brindar com brahma é um pa

In [44]:
print(generate_sentence("brindar com cerveja brahma é", 9))

brindar com cerveja brahma é é a prova viva de que deus nos ama
None
